In [20]:
library("GEOquery")
library("curl")
library('biomaRt')


# For cell types
geo_id  <- "GSE86469"
out_file <- "GSE86469_cell_features.tsv" # Will contain cell characteristics
out_file_dia <- "GSE86469_cell_features_diabetes.tsv" # Will contain cell characteristics
out_file_non_dia <- "GSE86469_cell_features_healthy.tsv" # Will contain cell characteristics

# Gene count matrix
url_count_mat <- "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE86nnn/GSE86469/suppl/GSE86469_GEO.islet.single.cell.processed.data.RSEM.raw.expected.counts.csv.gz"
out_file_count_mat <- "GSE86469_GEO.islet.single.cell.processed.data.RSEM.raw.expected.counts.tsv"
out_file_count_mat_dia <- "GSE86469_GEO.islet.single.cell.processed.data.RSEM.raw.expected.counts.diabetes.tsv"
out_file_count_mat_non_dia <- "GSE86469_GEO.islet.single.cell.processed.data.RSEM.raw.expected.counts.healthy.tsv"

In [2]:
# Reading geo data
gsm <- getGEO(geo_id, GSEMatrix = T)

Found 1 file(s)
GSE86469_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
File stored at: 
/var/folders/37/f_hljrvj7291y8ygpf4r54wh0000gq/T//RtmplkuOgn/GPL18573.soft


In [3]:
# Shows available metadata
colnames(pData(phenoData(gsm[[1]])))

[1] "title"                   "geo_accession"          
 [3] "status"                  "submission_date"        
 [5] "last_update_date"        "type"                   
 [7] "channel_count"           "source_name_ch1"        
 [9] "organism_ch1"            "characteristics_ch1"    
[11] "characteristics_ch1.1"   "characteristics_ch1.2"  
[13] "characteristics_ch1.3"   "characteristics_ch1.4"  
[15] "characteristics_ch1.5"   "characteristics_ch1.6"  
[17] "characteristics_ch1.7"   "growth_protocol_ch1"    
[19] "molecule_ch1"            "extract_protocol_ch1"   
[21] "extract_protocol_ch1.1"  "taxid_ch1"              
[23] "data_processing"         "data_processing.1"      
[25] "data_processing.2"       "data_processing.3"      
[27] "data_processing.4"       "platform_id"            
[29] "contact_name"            "contact_email"          
[31] "contact_institute"       "contact_address"        
[33] "contact_city"            "contact_state"          
[35] "contact_zip/postal_code" "contact_country"        
[37] "data_row_count"          "instrument_model"       
[39] "library_selection"       "library_source"         
[41] "library_strategy"        "relation"               
[43] "relation.1"              "supplementary_file_1"   
[45] "age:ch1"                 "bmi:ch1"                
[47] "cell type:ch1"           "disease:ch1"            
[49] "islet unos id:ch1"       "race:ch1"               
[51] "Sex:ch1"                 "tissue:ch1"

In [37]:
# Getting metadata
cols <- c("geo_accession", "title", "characteristics_ch1", paste0("characteristics_ch1.", 1:7))
nCols <- length(cols)
metadata <- pData(phenoData(gsm[[1]]))[, cols]

# Renaming to readeble names
cell_chars  <- sapply(metadata[1,4:nCols], as.character)
cols_rename <- c("geo", "cellID", "cellType", sapply(strsplit(cell_chars, ":"), function(x) x[[1]]))
colnames(metadata) <- cols_rename
                                           
# Cleaning certain values
for(i in 3:nCols) 
    metadata[,i] <- sapply(strsplit(as.character(metadata[,i, drop=T]), ": "), function(x) x[[2]])

# Changing the cell type ontology
correspondance <- setNames(c('unsure', 'beta', 'stellate', 'ductal', 'alpha', 'acinar', 'gamma', 'delta'),
                           c('None/Other', 'Beta', 'Stellate', 'Ductal', 'Alpha', 'Acinar', 'Gamma/PP', 'Delta'))
metadata[,'cellType'] <- correspondance[as.character(metadata[,'cellType'])]

# Separating between diabetes and non-diabetes
metadata_dia <- metadata[metadata$disease == "Type 2 Diabetic",]
metadata_non_dia <- metadata[metadata$disease == "Non-Diabetic",]
head(metadata)
head(metadata_dia)
head(metadata_non_dia)


[1] "unsure"   "beta"     "stellate" "ductal"   "alpha"    "acinar"   "gamma"   
[8] "delta"

,geo,cellID,cellType,tissue,Sex,disease,age,race,bmi,islet unos id
,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2303146,GSM2303146,10th_C10_S104,unsure,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303147,GSM2303147,10th_C11_S96,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303148,GSM2303148,10th_C13_S61,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303149,GSM2303149,10th_C14_S53,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303150,GSM2303150,10th_C16_S105,unsure,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303151,GSM2303151,10th_C17_S97,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009


,geo,cellID,cellType,tissue,Sex,disease,age,race,bmi,islet unos id
,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2303146,GSM2303146,10th_C10_S104,unsure,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303147,GSM2303147,10th_C11_S96,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303148,GSM2303148,10th_C13_S61,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303149,GSM2303149,10th_C14_S53,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303150,GSM2303150,10th_C16_S105,unsure,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009
GSM2303151,GSM2303151,10th_C17_S97,beta,Pancreatic Islet,Female,Type 2 Diabetic,55,White,29.8,ACIW009


,geo,cellID,cellType,tissue,Sex,disease,age,race,bmi,islet unos id
,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2303347,GSM2303347,1st-61_S27,beta,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268
GSM2303348,GSM2303348,1st-C11_S58,beta,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268
GSM2303349,GSM2303349,1st-C13_S19,beta,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268
GSM2303350,GSM2303350,1st-C15_S3,alpha,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268
GSM2303351,GSM2303351,1st-C18_S51,alpha,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268
GSM2303352,GSM2303352,1st-C19_S20,alpha,Pancreatic Islet,Male,Non-Diabetic,22,African American,32.95,ACCG268


In [38]:
# Save data
metadata <- t(metadata)
metadata_dia <- t(metadata_dia)
metadata_non_dia <- t(metadata_non_dia)

write.table(metadata, out_file, sep = "\t", quote = F, col.names = F)
write.table(metadata_dia, out_file_dia, sep = "\t", quote = F, col.names = F)
write.table(metadata_non_dia, out_file_non_dia, sep = "\t", quote = F, col.names = F)

In [39]:
# Now downloading and rearraging gene by cell counts

# Getting count matrix
con <- gzcon(url(url_count_mat))
count_mat <- readLines(con)
count_mat <- read.csv(textConnection(count_mat), check.names =F)
#colnames(count_mat)[-1] <- gsub("^X", "", colnames(count_mat)[-1])
close(con)

In [7]:
# Getting genes names
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
G_list <- getBM(filters= "ensembl_gene_id", 
                attributes= c("ensembl_gene_id","hgnc_symbol", "description"), 
                values=count_mat[,1], 
                mart= mart)

head(G_list)

ensembl_gene_id,hgnc_symbol,description
<chr>,<chr>,<chr>
ENSG00000011454,RABGAP1,RAB GTPase activating protein 1 [Source:HGNC Symbol;Acc:HGNC:17155]
ENSG00000022277,RTF2,replication termination factor 2 [Source:HGNC Symbol;Acc:HGNC:15890]
ENSG00000054796,SPO11,SPO11 initiator of meiotic double stranded breaks [Source:HGNC Symbol;Acc:HGNC:11250]
ENSG00000067900,ROCK1,Rho associated coiled-coil containing protein kinase 1 [Source:HGNC Symbol;Acc:HGNC:10251]
ENSG00000074410,CA12,carbonic anhydrase 12 [Source:HGNC Symbol;Acc:HGNC:1371]
ENSG00000078043,PIAS2,protein inhibitor of activated STAT 2 [Source:HGNC Symbol;Acc:HGNC:17311]


In [40]:
# Swapping ensembl ids to gene names
G_list <- G_list[!duplicated(G_list$ensembl_gene_id),]
rownames(G_list) <- G_list$ensembl_gene_id

count_mat[,1] <- as.character(count_mat[,1])
colnames(count_mat)[1] <- 'Gene'
gene_names <- G_list[count_mat[,1],2]
count_mat[!is.na(gene_names),1] <- gene_names[!is.na(gene_names)]

In [42]:
# Separate between diabetes and non-diabates
count_mat <- count_mat[,c("Gene", as.character(metadata["cellID",]))]
count_mat_dia <- count_mat[,c("Gene", as.character(metadata_dia["cellID",]))]
count_mat_non_dia <- count_mat[,c("Gene",as.character(metadata_non_dia["cellID",]))]

In [43]:
# Save data
write.table(count_mat, out_file_count_mat, sep = "\t", quote = F, col.names = T, row.names = F)
write.table(count_mat_dia, out_file_count_mat_dia, sep = "\t", quote = F, col.names = T, row.names = F)
write.table(count_mat_non_dia, out_file_count_mat_non_dia, sep = "\t", quote = F, col.names = T, row.names = F)